<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/3_submit_test/hj-submit_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tutorial/3_submit_test/hj-submit_test.ipynb

# Submit Test

Test Set은 30일에 3번만 제출 가능하지만, Validation Set은 무제한으로 제출 가능하다. Validationo Set으로 Submit Test 해보려고 한다.

아래 명령은 Google Storage에 있는 Waymo Open Dataset을 구글 드라이브로 다운받는 명령어다. Validation Set만 약 74GB이므로 유료 결제한 재승이 구글 드라이브에 올리고 공유해서 사용해보려고 한다.
```
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
drive.mount('/content/drive', force_remount=True)

!gsutil -m cp -r \
  "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/tf_example/validation" \
  /content/drive/MyDrive/WaymoDataset/Validation/
```

- 구글 드라이브 연동

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
VALIDATION_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed/scenario/validation/"

## Package Installaion

In [4]:
%%capture
!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1 # 최신 버전 라이브러리

## Imports and global definitions

In [2]:
import math
import os
import uuid
import time

import numpy as np
import tensorflow as tf

from google.protobuf import text_format
from waymo_open_dataset.metrics.ops import py_metrics_ops
from waymo_open_dataset.metrics.python import config_util_py as config_util
from waymo_open_dataset.protos import motion_metrics_pb2

In [3]:
file_paths = tf.io.gfile.glob(VALIDATION_PATH + "*.tfrecord*")

## Load motion prediction dataset

In [7]:
from waymo_open_dataset.protos import scenario_pb2 

dataset = tf.data.TFRecordDataset(file_paths, compression_type='')
for data in dataset :
   scenario = scenario_pb2.Scenario()
   scenario.ParseFromString(data.numpy())
   break

In [8]:
scenario.scenario_id

'a3cfb7f093ccd29'

In [9]:
scenario.tracks[1].id

1218

In [10]:
scenario.tracks[0].states[0].heading

-0.4103038012981415

In [11]:
len(scenario.tracks[1].states)

91

In [27]:
test = []
for i in range(len(scenario.tracks_to_predict)) :
  test.append(scenario.tracks_to_predict[i].track_index)

test

[1, 26, 0]

In [34]:
len([scenario.tracks[i] for i in test])

3

## 제출 형태 (0으로 채우기)

MotionChallengeSubmission 구조
```javascript
message MotionChallengeSubmission {
  enum SubmissionType {
    UNKNOWN = 0;

    // A submission for the Waymo open dataset motion prediction challenge.
    MOTION_PREDICTION = 1;

    // A submission for the Waymo open dataset interaction prediction challenge.
    INTERACTION_PREDICTION = 2;
  }

  // This must be set as the full email used to register at waymo.com/open.
  optional string account_name = 3;

  // This name needs to be short, descriptive and unique. Only the latest result
  // of the method from a user will show up on the leaderboard.
  optional string unique_method_name = 4;

  // Author information.
  repeated string authors = 5;
  optional string affiliation = 6;

  // A brief description of the method.
  optional string description = 7;

  // Link to paper or other link that describes the method.
  optional string method_link = 8;

  // The challenge submission type.
  optional SubmissionType submission_type = 2;

  // Set this to true if your model uses the lidar data provided in the motion
  // dataset.
  optional bool uses_lidar_data = 9;

  // The set of scenario predictions to evaluate. One entry should exist for
  // every record in the test set.
  repeated ChallengeScenarioPredictions scenario_predictions = 1;
}
```

In [4]:
from waymo_open_dataset.protos import motion_submission_pb2

submission = motion_submission_pb2.MotionChallengeSubmission()

In [13]:
[e for e in submission.__dir__() if '_' != e[0]]

['DESCRIPTOR',
 'account_name',
 'unique_method_name',
 'authors',
 'affiliation',
 'description',
 'method_link',
 'submission_type',
 'uses_lidar_data',
 'scenario_predictions',
 'SubmissionType',
 'UNKNOWN',
 'MOTION_PREDICTION',
 'INTERACTION_PREDICTION',
 'ByteSize',
 'Clear',
 'ClearExtension',
 'ClearField',
 'CopyFrom',
 'DiscardUnknownFields',
 'FindInitializationErrors',
 'FromString',
 'HasExtension',
 'HasField',
 'IsInitialized',
 'ListFields',
 'MergeFrom',
 'MergeFromString',
 'ParseFromString',
 'RegisterExtension',
 'SerializePartialToString',
 'SerializeToString',
 'SetInParent',
 'UnknownFields',
 'WhichOneof',
 'Extensions']

In [5]:
from waymo_open_dataset.protos import scenario_pb2 

scenarios = tf.data.TFRecordDataset(file_paths, compression_type='')

In [7]:
for s in scenarios :
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(s.numpy())
  break

In [17]:
scenario.timestamps_seconds

[0.0, 0.10003, 0.20012, 0.30018, 0.40024, 0.5003, 0.60036, 0.70043, 0.80052, 0.90062, 1.00069, 1.10079, 1.20092, 1.30102, 1.40112, 1.5013, 1.6014, 1.70153, 1.80167, 1.90178, 2.00188, 2.10197, 2.20207, 2.30217, 2.40227, 2.50236, 2.60246, 2.70256, 2.80266, 2.90271, 3.00282, 3.10288, 3.20295, 3.30301, 3.40307, 3.50316, 3.60322, 3.70328, 3.80335, 3.90341, 4.00343, 4.10345, 4.20351, 4.30354, 4.4036, 4.50366, 4.60375, 4.7038, 4.80383, 4.90389, 5.00396, 5.10405, 5.20415, 5.3042, 5.40426, 5.50432, 5.60442, 5.70448, 5.80454, 5.90457, 6.00463, 6.10465, 6.20471, 6.30474, 6.4048, 6.50486, 6.60492, 6.70498, 6.80505, 6.90511, 7.00518, 7.10524, 7.2053, 7.30537, 7.40542, 7.50549, 7.60555, 7.70561, 7.80567, 7.90573, 8.00576, 8.10579, 8.20581, 8.30579, 8.40582, 8.50584, 8.60585, 8.70588, 8.8059, 8.90593, 9.00594]

In [24]:
count = 0
for i in scenario.tracks[0].states[10:] :
  count += 1
count

81

In [32]:
from waymo_open_dataset.protos import scenario_pb2 

scenarios = tf.data.TFRecordDataset(file_paths, compression_type='')
for i, s in enumerate(scenarios) :
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(s.numpy())
  scenario_predictions = submission.scenario_predictions.add()
  scenario_predictions.scenario_id = scenario.scenario_id
  prediction_set = scenario_predictions.single_predictions

  print(str(i), ' : ', scenario.scenario_id)
  

  track_index = []
  for i in range(len(scenario.tracks_to_predict)) :
    track_index.append(scenario.tracks_to_predict[i].track_index)
  
  for track in [scenario.tracks[i] for i in track_index] :
    predictions = prediction_set.predictions.add()
    predictions.object_id = int(track.id)

    '''
    y = 0
    rot_matrix = np.array([
        [np.cos(y), -np.sin(y)],
        [np.sin(y), np.cos(y)],
    ])
    '''
    current_time_index = scenario.current_time_index + 1
    for i in track.states[current_time_index:] :
      scored_trajectory = predictions.trajectories.add()

      trajectory = scored_trajectory.trajectory

      trajectory.center_x.append(i.center_x)
      trajectory.center_y.append(i.center_y) 




스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
39097  :  4a1dc018bb31da91
39098  :  df8dcf4e1ba1236
39099  :  fcb7feccb308c387
39100  :  fe4ba13d1d3f8a66
39101  :  4065dc7139c718d8
39102  :  1ec313c088fc8993
39103  :  f91f02b501a02ebc
39104  :  d94756db7f7f20a5
39105  :  f246f13f819c7f3f
39106  :  60d3992596e61dae
39107  :  1d23d8d8d8a1f9ef
39108  :  8a782bb99611655e
39109  :  ae406651c6f2ad1c
39110  :  71fc94cc4bf5099b
39111  :  de75d5cd8bdcf0c
39112  :  205f8b7c7c892a50
39113  :  7b3a0e3a459a74eb
39114  :  8752012bafbb28d0
39115  :  550fb9aba453daca
39116  :  12b43024d8625d9
39117  :  fc922243e3e1d592
39118  :  24785c3b1ae5a38f
39119  :  a4054101f562ac9c
39120  :  2aff3ac2e4f49612
39121  :  a87f085a3043f4ea
39122  :  7ede14b919eafdab
39123  :  6c222820756391cb
39124  :  c1ed40c9ebf27d91
39125  :  58d61ed5dfe2c174
39126  :  4fede1f1c63a09c1
39127  :  da5553e8f321711e
39128  :  1e26ad1e0d7fbc3a
39129  :  80d3e686eb89c0d0
39130  :  1043637618595374
39131  :  b42d153f9f7b5fde
39132  :  c198e3158bbc

tracks_to_predict에 해당하는 tracks만 입력했다. 실행 시간은 24분 걸렸다.

In [33]:
submission.account_name = 'hyocurity@gmail.com'
submission.unique_method_name = 'HJNet'
submission.authors.append('LEE HYOJIN')
submission.affiliation = 'LOAD'
submission.description = "Submission Test (val set)."
submission.method_link = 'NA'
submission.submission_type = submission.MOTION_PREDICTION
submission.uses_lidar_data = False

In [34]:
import tarfile
from time import time
import io
from google.colab import files

timestamp = str(time()).split('.')[0]
fname = f'{timestamp}-motion-pred-submission.tar.gz'

# submission 파일의 형태는 bin이지만, tarball(tar.gz)로 압축해서 제출해야 함.
with tarfile.open(fname, "w:gz") as tar:
    f = io.BytesIO(submission.SerializeToString())
    info = tarfile.TarInfo(name=fname)
    info.size = len(f.getvalue())
    tar.addfile(tarinfo=info, fileobj=f)

files.download(fname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

제출했을 때 채점이 15분 정도 걸렸고 `INVALID_ARGUMENT: Duplicate prediction scenario ID found : 17c4e1e5a9665445` 에러 문구가 떴다. 

scenario_id 중복값이 생기는 문제를 해결해야 한다.

In [35]:
id = []
for n in range(len(submission.scenario_predictions)) :
  id.append(submission.scenario_predictions[n].scenario_id)
id[0]

'a3cfb7f093ccd29'

In [47]:
count = 0
for t in scenarios :
  count += 1
count

44097

In [38]:
len(id)

44447

In [36]:
dup = [x for i, x in enumerate(id) if x in id[:i]]

In [62]:
for i, s in enumerate(scenarios) :
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(s.numpy())
  print(scenario.scenario_id)
  if i == 20 :
    break


a3cfb7f093ccd29
defde13be96c21e4
a672051090992916
a540704af740fe42
a431137a9fa4a1db
8dfefbaed04192fb
73da1e6dc53d926b
d958dfe73371e86f
855028a6d78335e5
17c4e1e5a9665445
6f40dcfe056310ef
e21e441beffa2a08
448be4c3a722913a
e71a3e9e66df7653
f9937b8881aa4c40
885e6a49613afd02
e37da7bbb9a02389
f880fa6dadcfff33
de1e15269c8d2b45
8fd6a3f983db579f
6796b7c3eef5c74d


In [40]:
len(dup)

350